In [22]:
import os
import sys
import time
import datetime
import numpy as np
import pandas as pd
import pathlib
from collections import Counter, defaultdict
from pprint import pprint
from io import StringIO

from bs4 import BeautifulSoup
from xml.dom import minidom
from xml.etree import ElementTree as ET
#from lxml import etree as ET #Supports xpath syntax

#https://www.datacamp.com/community/tutorials/python-xml-elementtree
#https://stackabuse.com/reading-and-writing-xml-files-in-python/
#https://codereview.stackexchange.com/questions/117810/parsing-of-xml-in-python-using-minidom-and-inserting-data-into-a-database
#https://stackoverflow.com/questions/749796/pretty-printing-xml-in-python

In [33]:
pwd

'C:\\Users\\rsouza\\Documents\\Python Scripts\\exploreAT-Topothek.NLP'

Reading the XML Metadata files:

In [37]:
readme =  './TEI-XML-2018/README_English.txt'
print((open(readme, encoding="latin-1").read()))


Content of the XML files
XML are partially enriched with location data
* The location data are available as a tree in the respective entries
* Location data are available for those entries specified in helper_tables\placeName-bis Freq 10-nur Ö+STir - Tabellenblatt1.csv
* The basis are complete matches in the tag placeName

The complete place hierarchy can be found in helper_tables\listPlace.xml. This resulted in helper_tables\listPlaceBySigle.xml

There could be manual corrections to the data.


In [41]:
placesfile =  './TEI-XML-2018/helper_tables/listPlace.xml'
#print((open(placesfile, encoding="utf-8").read()))
placestree = ET.parse(placesfile)
placesroot = placestree.getroot()
set([elem.tag for elem in placesroot.iter()])

{'{http://www.tei-c.org/ns/1.0}Bundesland',
 '{http://www.tei-c.org/ns/1.0}Gemeinde',
 '{http://www.tei-c.org/ns/1.0}Großregion',
 '{http://www.tei-c.org/ns/1.0}Kleinregion',
 '{http://www.tei-c.org/ns/1.0}Kommentar',
 '{http://www.tei-c.org/ns/1.0}Ort',
 '{http://www.tei-c.org/ns/1.0}Sigle_DB',
 '{http://www.tei-c.org/ns/1.0}idno',
 '{http://www.tei-c.org/ns/1.0}listPlace',
 '{http://www.tei-c.org/ns/1.0}note',
 '{http://www.tei-c.org/ns/1.0}place',
 '{http://www.tei-c.org/ns/1.0}placeName',
 '{http://www.tei-c.org/ns/1.0}record'}

Reading the XML Data files:

In [2]:
xmlfile =  './TEI-XML-2018/bel_d-tei-02/d157_qdb-TEI-02.xml'
#xmlfile = open(xmlfile, mode='r', encoding='utf-8').read()
namespaces = {'TEI': 'http://www.tei-c.org/ns/1.0'} # add more as needed

Parsing the XML tree

In [9]:
xmldoc = minidom.parse(xmlfile)  #xmldoc = ET.XML(xmlfile)
bs = BeautifulSoup(open(xmlfile, encoding="utf8"), 'xml')
tree = ET.parse(xmlfile)

In [10]:
root = tree.getroot()
print(root)

<Element '{http://www.tei-c.org/ns/1.0}TEI' at 0x000001D838907548>


In [11]:
my_namespaces = dict([node for _, node in ET.iterparse(StringIO(open(xmlfile, mode='r', encoding='utf-8').read()), events=['start-ns'])])
print(my_namespaces)

{'': 'http://www.tei-c.org/ns/1.0'}


In [15]:
for child in root:
    print(child.tag, child.attrib)

{http://www.tei-c.org/ns/1.0}teiHeader {}
{http://www.tei-c.org/ns/1.0}text {}


In [17]:
set([elem.tag for elem in root.iter()])

{'{http://www.tei-c.org/ns/1.0}TEI',
 '{http://www.tei-c.org/ns/1.0}att',
 '{http://www.tei-c.org/ns/1.0}bibl',
 '{http://www.tei-c.org/ns/1.0}body',
 '{http://www.tei-c.org/ns/1.0}certainty',
 '{http://www.tei-c.org/ns/1.0}cit',
 '{http://www.tei-c.org/ns/1.0}date',
 '{http://www.tei-c.org/ns/1.0}def',
 '{http://www.tei-c.org/ns/1.0}edition',
 '{http://www.tei-c.org/ns/1.0}editionStmt',
 '{http://www.tei-c.org/ns/1.0}editorialDecl',
 '{http://www.tei-c.org/ns/1.0}encodingDesc',
 '{http://www.tei-c.org/ns/1.0}entry',
 '{http://www.tei-c.org/ns/1.0}etym',
 '{http://www.tei-c.org/ns/1.0}fileDesc',
 '{http://www.tei-c.org/ns/1.0}form',
 '{http://www.tei-c.org/ns/1.0}geoDecl',
 '{http://www.tei-c.org/ns/1.0}gi',
 '{http://www.tei-c.org/ns/1.0}gram',
 '{http://www.tei-c.org/ns/1.0}gramGrp',
 '{http://www.tei-c.org/ns/1.0}idno',
 '{http://www.tei-c.org/ns/1.0}interp',
 '{http://www.tei-c.org/ns/1.0}label',
 '{http://www.tei-c.org/ns/1.0}listPlace',
 '{http://www.tei-c.org/ns/1.0}listPrefixDe

In [29]:
places = []
for elem in root.iter('{http://www.tei-c.org/ns/1.0}place'):
    # print(elem.attrib)
    # print(elem.attrib['type'])
    places.append(elem.attrib['type'])
freq_places = Counter(places)
freq_places.most_common()

[('Bundesland', 891),
 ('Großregion', 797),
 ('Kleinregion', 783),
 ('Gemeinde', 696),
 ('Ort', 696)]

In [31]:
for elem in root.iter('{http://www.tei-c.org/ns/1.0}cit'):
    print(elem.attrib)

{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '2'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '2'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '2'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '2'}
{'type': '